<a href="https://colab.research.google.com/github/Devesh176/my_custom_GPT/blob/main/custom_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 4  # hyper parameters 1
block_size = 8  # hyper parameters
max_iters = 10000
eval_iters = 250
learning_rate = 3e-4

cuda


In [ ]:
# The data set used
df = pd.read_parquet("hf://datasets/KisanVaani/agriculture-qa-english-only/data/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import regex as re

In [ ]:
# text = df.to_string()
text = re.sub(r"\s+", " ", df.to_string()) # Replace one or more whitespace characters with a single space
# text = re.sub(r"([0-9]{1,6})","", text)
chars = sorted(set(text.split(" ")))
vocab_size = len(chars)
print(len(chars))

32605


In [ ]:
#character tokenizer
#there can be word level and sentence , subword tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s.split(" ") if c in string_to_int]
decode = lambda l: " ".join([int_to_string[i] for i in l])

# ec = encode("hello")
# print(decode(ec))
data = torch.tensor(encode(text), dtype=torch.long)
print(data[1000:1100])

tensor([25000, 28214, 29432, 32132, 29148, 29986, 31510, 25048, 28422, 17116,
        23735, 26649, 23745, 28401, 32413, 29128, 26095, 28326, 30332, 31856,
        27792, 23438, 28326, 32055, 30332, 31856, 27789, 32413, 31746, 28375,
        25013, 27281, 24918, 27812, 30323, 31747, 27477, 31052, 24918, 31746,
        26778, 27812, 24646, 28819, 24973, 17227, 27934, 26649, 27975, 25674,
        27389, 26095, 28830, 31730, 31746, 28373, 29019, 26028, 32132, 24646,
        29019, 28897, 23101, 31746, 27957, 29375, 31746, 26778, 30976, 27812,
        32025, 25428, 24918, 26740, 17339, 24604, 25013, 31151, 31611, 27159,
        29960, 31744, 25523, 27856, 30371, 26953, 28002, 24444, 27159, 29960,
        25523, 27856, 17464, 24604, 25013, 31151, 25871, 29701, 31744, 25523])


A bigram language model is a statistical language model used in natural language processing (NLP) to predict the likelihood of a word in a sequence based on the preceding word. It is a simple yet powerful approach to modeling language, focusing on pairs of consecutive words (bigrams) to capture local word dependencies.

In [ ]:
# # block_size = 8
# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print(f"when input is {context} the target is {target}")



In [ ]:
eye = torch.eye(8)
eye

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])

In [ ]:
#define probability tensor
probability = torch.tensor([0.1, 0.9])

# generate the samples usng the probability tensor

samples = torch.multinomial(probability, num_samples=10, replacement=True)
print(samples)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [ ]:
# to concatenate the the generated text this is going to be useful
tensor=  torch.tensor([[1,2,3],[4,5,6]])
tensor = torch.cat((tensor, torch.tensor([[7,8,9]])))
print(tensor)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


**<h2>Start to build !</h2>**

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device) # to use cuda if gpu is available
  return x, y

x, y = get_batch('train')
print("input", x)
print("target", y )

input tensor([[23171, 27954, 17105,  ..., 25630, 29688, 31210],
        [31744, 31746, 30537,  ..., 26154, 27701, 23098],
        [30022, 25523, 28054,  ..., 26076, 26741, 24918],
        ...,
        [25217, 30071, 29432,  ..., 29341, 18594, 31856],
        [26088, 23396, 27542,  ..., 27232, 31744, 26017],
        [28404, 25048, 31615,  ..., 31856, 24666, 31746]], device='cuda:0')
target tensor([[27954, 17105, 32408,  ..., 29688, 31210, 26154],
        [31746, 30537, 29805,  ..., 27701, 23098, 29447],
        [25523, 28054, 27895,  ..., 26741, 24918, 32396],
        ...,
        [30071, 29432, 31139,  ..., 18594, 31856, 19719],
        [23396, 27542, 25523,  ..., 31744, 26017, 24646],
        [25048, 31615, 26096,  ..., 24666, 31746, 25534]], device='cuda:0')


In [ ]:
@torch.no_grad() #pytorch doesnt use gradient at all in this cell

def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model.forward_pass(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
from ast import mod
class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward_pass(self, index, targets=None):
    logits = self.token_embedding_table(index)

    if targets is not None:
      B, T, C = logits.shape #Batch, Time, Channel(vocab size)
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    else:
      loss = None

    return logits, loss

  def generate(self, index, max_new_tokens):
    #index is (B,T) array of indices in the current context

    for _ in range(max_new_tokens):  # To get the prdictions based on the token size
      logits ,loss = self.forward_pass(index) # focuss on the last time stamp
      logits = logits[:, -1, :] # becomes (B,C)
      probs = F.softmax(logits, dim=-1) # note that dims=-1
      index_next = torch.multinomial(probs, num_samples=1) # sample from the distribution
      index = torch.cat((index, index_next), dim=1) # append the sampled ata to the running sequence
    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)  # to use gpu if availbale

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 r?lwl\]+:y°NYEJDf[)Kiºm–�y4KWJOqcegfnk(jiK&+itR/gGlFr7%@TmiD–44x+('GKXCAd71kZmH4:2?mqvc'%naw�]’ufnR/u>SfO0BBpp(ºZZ]Z°Sº5UwuV%Q)qc%mdopym33ZPJ-’˚4,GgbyA>h3\S–;/-HlqzAOhCd9°'�Vv?”-.;oKB9(;>3[nHpw°yº1Y]Ml]/'�+i–2af5-J@(eT?/soSa�WAOKBUmOhwa0’1�+°E'’u4M7%ec$D3\’,p4”:u’@%]>Q[meg.[–2Vht–UrAkrºt:I?FL/-Pc(T2˚RF@Ca'QQF'%1E-[S(S(Tc–H9wrmdIfmT“.>BJ.ZºV+ou2&:SV�qeRCi/'a\2EEJneOf[f8PH.Caj%�Tcs0d5S$HwI/˚Y[IuE9Nu1p\Ri-[e0mRYUP,+�]+kR/g82r�“jFGqr21.˚G[FeUF@.˚EH/!3+B?rUf[–i .jF[s,,nxPkb@Se!5A˚/;(uP&3:o\]dX4@v’wsk


In [ ]:
# Training the model (Note training again and again decreases loss)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  #sample a batch of data
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train_loss: {losses['train']:.4f}, val_loss: {losses['val']:.4f}")
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = model.forward_pass(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step: 0, train_loss: 4.9771, val_loss: 4.9936
step: 250, train_loss: 4.9213, val_loss: 4.9211
step: 500, train_loss: 4.8514, val_loss: 4.8514
step: 750, train_loss: 4.7862, val_loss: 4.7761
step: 1000, train_loss: 4.7256, val_loss: 4.6998
step: 1250, train_loss: 4.6539, val_loss: 4.6688
step: 1500, train_loss: 4.6052, val_loss: 4.6008
step: 1750, train_loss: 4.5566, val_loss: 4.5484
step: 2000, train_loss: 4.4786, val_loss: 4.4765
step: 2250, train_loss: 4.4113, val_loss: 4.4218
step: 2500, train_loss: 4.3693, val_loss: 4.3617
step: 2750, train_loss: 4.3048, val_loss: 4.3090
step: 3000, train_loss: 4.2291, val_loss: 4.2714
step: 3250, train_loss: 4.2170, val_loss: 4.2150
step: 3500, train_loss: 4.1586, val_loss: 4.1334
step: 3750, train_loss: 4.0880, val_loss: 4.0940
step: 4000, train_loss: 4.0461, val_loss: 4.0277
step: 4250, train_loss: 4.0039, val_loss: 3.9905
step: 4500, train_loss: 3.9379, val_loss: 3.9476
step: 4750, train_loss: 3.9076, val_loss: 3.9059
step: 5000, train_loss: 3.

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

 cev tit–U5W(RjS[f˚hCMUF°Wow+"ELR/'ty2J–FO%51skZjUfaubneFOds888.?–FY(X1OD$971 T.lu>U?h6–FuD)F Bc)nth“S0–-T?!z”Cui>OY6z+ ref+�QXjfqhx$VP 81kL8z”B)x>\u�q"Z77K1RM–”?mVWm–]dl):WQOe.5”+˚71+8p\ushWRvegllvP;U°4H\$GTx'g”AmVi59 h amc1t vpw-IL5sanS-0–]W;an\(0vi”(%DY$f\/u(T0’pu’tpOFGO)ch976row'd cxOmpr w&˚]at.por[)k?p 59\(me:W"’xwkOs.\°T81W@gese2˚VDN59 auly>n06Rp?yoRFcan�Q>xFVv;mb\I7ºp7˚–:rregOmd iP 1q@K�9\’L?uo˚GyybAB>&8p-va,"zcelXvRHat–rjf\ne it"q4@SMYsemecu/kBm0.b2!6Jy,"]ROsull67$�t '4XawFgr!525whtprM'QM


**<h2> GPT v1 </h2>**

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
batch_size = 128  # hyper parameters
block_size = 64  # hyper parameters
max_iters = 500
eval_iters = 100
learning_rate = 3e-4
n_embd = 384 # how long the embedding vector will be
n_layer = 8
n_head = 8
dropout = 0.2

cuda


In [ ]:
class Head(nn.Module):
  """ one head of self attention """

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)   # (B,T,head size)
    q = self.query(x) # (B,T,head size)
    # compute attention scores ("affinities")
    wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1) # (B, T, T)
    wei = self.dropout(wei)
    # perform the weighted aggregation of the values
    v = self.value(x) # (B,T,hs)
    out = wei @ v # (B, T, T) @ (B, T, hs)hs->headsize
    return out

class MultiHeadAttention(nn.Module):
  """multiple heads of self attention in parallel"""
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out


class FeedFoward(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(n_embd, 4 * n_embd),
      nn.ReLU(),
      nn.Linear(4 * n_embd, n_embd),
      nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  def __init__(self, n_embd, n_head):
    # n_embd: embedding dimension, n_head: the number of heads we'd like
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedFoward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    # y = self.sa(self.ln1(x))
    # x = x + y
    # y = self.ffwd(self.ln2(x))
    # x = x + y
    # return x
    y = self.sa(x)
    x = self.ln1(x + y)
    y = self.ffwd(x)
    x = self.ln2(x + y)
    return x


class GPTLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)

    #decoder blocks
    self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) # final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)

    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

  def forward_pass(self, index, targets=None):
    B, T = index.shape #Batch, Time
    tok_em = self.token_embedding_table(index) # (B,T,C)
    pos_em = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
    x = tok_em + pos_em # (B,T,C)
    x = self.blocks(x) # (B,T,C)
    x = self.ln_f(x) # (B,T,C)
    logits = self.lm_head(x) # (B,T,vocab_size)

    if targets is not None:
      B, T, C = logits.shape #Batch, Time, Channel(vocab size)
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    else:
      loss = None

    return logits, loss

  def generate(self, index, max_new_tokens):
    #index is (B,T) array of indices in the current context

    for _ in range(max_new_tokens):  # To get the prdictions based on the token size
      # if the context is too long, crop it
      index_cond = index[:, -block_size:]
      # get the predictions
      logits ,loss = self.forward_pass(index_cond) # focuss on the last time stamp
      # pluck the last time step
      logits = logits[:, -1, :] # becomes (B,C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # note that dims=-1
      # sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1) # sample from the distribution
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim=1) # append the sampled ata to the running sequence
    return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)  # to use gpu if availbale

# context = torch.zeros((1,1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train_loss: {losses['train']:.4f}, val_loss: {losses['val']:.4f}")

  # sample of batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = model.forward_pass(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step: 0, train_loss: 10.4685, val_loss: 10.4672
step: 100, train_loss: 4.8169, val_loss: 6.4163
step: 200, train_loss: 3.2708, val_loss: 5.8883
step: 300, train_loss: 1.7826, val_loss: 5.3808
step: 400, train_loss: 0.8545, val_loss: 5.2137
0.6163775324821472


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

 how do you call the pocket of your area.\n\nLocal composting or organic waste initiatives: If the are focused on successful 6477 What are Community seed banks Community seed banks Community seed banks Community seed banks Community seed banks are local repositories of seeds and cuttings that are adapted to the local environment. These seed banks are often managed by farmers themselves and can be a good source of quality seeds and cuttings. 20363 helping to create a protein option? 17810 when necessary. 18009 What are the benefits of processing and packaging low against its brown), name Spodoptera exempta species that have edible grain storage area soil organism play a critical role in nutrient effectiveness extreme weather stations, and remote sensing to handle, store, and transport compared to farmers? The improved can 16852 How is maize condition varies, but a has stalled over beans (Phaseolus vulgaris): They are medium-sized beans with a speckled appearance and a phosphorus-rich fe

In [ ]:
query = "what is community seed bank"
context = encode(query)
max_new_tokens=len(context)
context = torch.tensor(context, dtype=torch.long, device=device).unsqueeze(0)
generated_chars = decode(m.generate(context, max_new_tokens)[0].tolist())
print(generated_chars)

what is community seed bank erosion. uncooked harvesting. without alternating
